### usual imports

In [ ]:
import os
import numpy as np
from rail.core.utils import RAILDIR
#from rail.estimation.algos.bpz_version.utils import RAIL_BPZ_DIR
from rail.pipelines.estimation.inform_all import InformPipeline
from rail.core import common_params
RAIL_BPZ_DIR = '../../../rail_bpz/rail_bpz/src/'

### The following two cells define common parameters before building stages. These parameters are shared by all the stages. Params changed in individual stages will not result in changes in common_params

In [ ]:
bands = 'grizy'
maglims = [27.66, 27.25, 26.6, 26.24, 25.35]
maglim_dict={}
for band,limx in zip(bands, maglims):
    maglim_dict[f"HSC{band}_cmodel_dered"] = limx

In [ ]:
common_params.set_param_defaults(
    bands=[f'HSC{band}_cmodel_dered' for band in bands],
    err_bands=[f'{band}_cmodel_magerr' for band in bands],
    nondetect_val=np.nan,
    ref_band='HSCi_cmodel_dered',
    redshift_col='specz_redshift',
    mag_limits=maglim_dict,
    zmax = 6.0,
)

from rail.core import *
RailEnv.print_rail_stage_dict()

### Inform the pipeline

In [ ]:
pipe = InformPipeline()

### Examples of updating parameters for individual stages

In [ ]:
#pipe.inform_knn.config.update(leaf_size=14)  #example to change config param values
pipe.inform_somoclu.config.update(gridtype = 'hexagonal', maptype="toroid", column_usage='colors', std_coeff=2, som_learning_rate=10)
pipe.inform_simplesom.config.update(column_usage='colors')

pipe.inform_bpz.config.update(
    zp_errors=[0.01, 0.01, 0.01, 0.01, 0.01],
    columns_file = os.path.join(
        RAIL_BPZ_DIR, 'rail', 'examples_data', 'estimation_data', 'configs', 'test_bpz_hsc.columns',
    ),
)

### Define the input file to inform stages

In [ ]:
DATA_DIR = "/net/home/fohlen14/yanza21/DATA/data_for_rail/"  # the directory that sore data
# DATA_DIR = "/global/cfs/cdirs/desc-pz/HSCDATA"  # the data directory on NERSC

In [ ]:
#input_file = '/net/home/fohlen14/yanza21/DATA/data_for_rail/dered_223501_sz_match_pdr3_dud_NONDET.hdf5'
#input_file = '/net/home/fohlen14/yanza21/DATA/data_for_rail/hecto_deredden_nondet.hdf5'
input_file = f'{DATA_DIR}GAMA15H_calibrated_pz_dered.hdf5'
input_dict = dict(
    input=input_file,
)

### Initialize the pipeline with the input file.
Setting `resume=True` if you want to skip pre-informed stages
None is an external configuration file that is not defined in the previous cells

In [ ]:
pipe_info = pipe.initialize(input_dict, dict(output_dir='.', log_dir='.', resume=False), None)  

### Save the configuration to a yaml file

In [ ]:
pipe.save('tmp_inform_all.yml')

### Read the configuration yaml file and run the whole inform pipeline

In [ ]:
import ceci
pr = ceci.Pipeline.read('tmp_inform_all.yml')

In [ ]:
pr.run()